In [1]:
# import dependencies
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import requests
from splinter import Browser
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()

In [2]:
# get draft data
engine = create_engine('sqlite:///../data/fbdata.sqlite')
Base.metadata.create_all(engine)
Base.prepare(engine, reflect=True)
session = Session(engine)
draftDf = pd.read_sql_query("SELECT DraftYear, Player, CollegeStats FROM draftHistory WHERE Pos = 'QB'", con=engine)
draftDf

,DraftYear,Player,CollegeStats
0,1987,Vinny Testaverde,http://www.sports-reference.com/cfb/players/vi...
1,1987,Kelly Stouffer,http://www.sports-reference.com/cfb/players/ke...
2,1987,Chris Miller,http://www.sports-reference.com/cfb/players/ch...
3,1987,Jim Harbaugh,http://www.sports-reference.com/cfb/players/ji...
4,1987,Cody Carlson,http://www.sports-reference.com/cfb/players/co...
...,...,...,...
430,2020,Jake Luton,http://www.sports-reference.com/cfb/players/ja...
431,2020,Cole McDonald,http://www.sports-reference.com/cfb/players/co...
432,2020,Ben DiNucci,http://www.sports-reference.com/cfb/players/be...
433,2020,Tommy Stevens,http://www.sports-reference.com/cfb/players/to...


In [3]:
# loop through links.
cfbStatsDf = pd.DataFrame(columns = ['year','school','conf','class','pos','games','rushAtt','rushYds','rushAvg','rushTD','rec',\
                               'recYds','recAvg','recTD','plays','ydsFromScrim','ydsFromScrimAvg','rushRecTD'])
for index, row in draftDf.iterrows():
    if row.CollegeStats:
        print(row.CollegeStats)
        soup = bs(requests.get(row.CollegeStats).text, 'lxml')
        try:
            tables = soup.findAll(class_='table_wrapper')
            all_passing = pd.read_html(soup.find(id=f"all_passing").prettify(), header=1)[0]
            all_passing.columns = ['year','school','conf','class','pos','games','passComp','passAtt','compPct','passYds',\
                                   'ydsPerAtt','adjYdsPerAtt','TD','int','effRate']
            all_passing.year = all_passing.year.str.strip('*').str.strip(' ')
            all_rushing = pd.read_html(soup.find(id=f"all_rushing").findAll(text=lambda text:isinstance(text, Comment))[0].extract(), header=1)[0]
            all_rushing.columns = ['year','school','conf','class','pos','games','rushAtt','rushYds','rushAvg','rushTD','rec',\
                                   'recYds','recAvg','recTD','plays','ydsFromScrim','ydsFromScrimAvg','rushRecTD']
            all_rushing.year = all_passing.year.str.strip('*').str.strip(' ')
            # all_passing.head()
            playerdf = pd.merge(all_passing, all_rushing, on=['year','school','conf','class','pos','games'])
            playerdf['name'] = row.Player
            playerdf['DraftYear'] = row.DraftYear
            cfbStatsDf = cfbStatsDf.append(playerdf, ignore_index=True)
        except:
            print('fail')
        

http://www.sports-reference.com/cfb/players/vinny-testaverde-1.html
http://www.sports-reference.com/cfb/players/kelly-stouffer-1.html
http://www.sports-reference.com/cfb/players/chris-miller-4.html
http://www.sports-reference.com/cfb/players/jim-harbaugh-1.html
http://www.sports-reference.com/cfb/players/cody-carlson-1.html
http://www.sports-reference.com/cfb/players/mark-vlasic-1.html
http://www.sports-reference.com/cfb/players/steve-beuerlein-1.html
http://www.sports-reference.com/cfb/players/kevin-sweeney-1.html
http://www.sports-reference.com/cfb/players/sammy-garza-1.html
http://www.sports-reference.com/cfb/players/don-majkowski-1.html
http://www.sports-reference.com/cfb/players/john-paye-2.html
http://www.sports-reference.com/cfb/players/mike-shula-1.html
http://www.sports-reference.com/cfb/players/jim-karsatos-1.html
http://www.sports-reference.com/cfb/players/bill-ransdell-2.html
http://www.sports-reference.com/cfb/players/tom-tupa-1.html
http://www.sports-reference.com/cfb/pla

http://www.sports-reference.com/cfb/players/jonathan-quinn-1.html
http://www.sports-reference.com/cfb/players/brian-griese-1.html
http://www.sports-reference.com/cfb/players/john-dutton-2.html
http://www.sports-reference.com/cfb/players/matt-hasselbeck-1.html
http://www.sports-reference.com/cfb/players/moses-moreno-1.html
http://www.sports-reference.com/cfb/players/tim-couch-1.html
http://www.sports-reference.com/cfb/players/donovan-mcnabb-1.html
http://www.sports-reference.com/cfb/players/akili-smith-1.html
http://www.sports-reference.com/cfb/players/daunte-culpepper-1.html
http://www.sports-reference.com/cfb/players/cade-mcnown-1.html
http://www.sports-reference.com/cfb/players/shaun-king-1.html
http://www.sports-reference.com/cfb/players/brock-huard-1.html
http://www.sports-reference.com/cfb/players/joe-germaine-1.html
http://www.sports-reference.com/cfb/players/aaron-brooks-1.html
http://www.sports-reference.com/cfb/players/michael-bishop-1.html
http://www.sports-reference.com/cfb/

http://www.sports-reference.com/cfb/players/stephen-mcgee-1.html
http://www.sports-reference.com/cfb/players/tom-brandstater-1.html
http://www.sports-reference.com/cfb/players/curtis-painter-1.html
http://www.sports-reference.com/cfb/players/sam-bradford-1.html
http://www.sports-reference.com/cfb/players/tim-tebow-1.html
http://www.sports-reference.com/cfb/players/jimmy-clausen-1.html
http://www.sports-reference.com/cfb/players/colt-mccoy-1.html
http://www.sports-reference.com/cfb/players/mike-kafka-1.html
http://www.sports-reference.com/cfb/players/rusty-smith-1.html
http://www.sports-reference.com/cfb/players/tony-pike-1.html
http://www.sports-reference.com/cfb/players/levi-brown-1.html
http://www.sports-reference.com/cfb/players/cam-newton-1.html
http://www.sports-reference.com/cfb/players/jake-locker-1.html
http://www.sports-reference.com/cfb/players/blaine-gabbert-1.html
http://www.sports-reference.com/cfb/players/christian-ponder-1.html
http://www.sports-reference.com/cfb/players

In [4]:
# get total number of games on career row
totgames = 0
for index, row in cfbStatsDf.iterrows():
    if row.year == 'Career':
        cfbStatsDf.at[index, 'games'] = totgames
        totgames = 0;
    else:
        if row.games > 0:
            totgames += row.games

In [5]:
# get player data
cfbCareerStatsDf = cfbStatsDf[cfbStatsDf['year'] == 'Career']
cfbYearStatsDf = cfbStatsDf[cfbStatsDf['year'] != 'Career']
del cfbCareerStatsDf['conf']
del cfbCareerStatsDf['pos']
del cfbCareerStatsDf['class']
cfbCareerStatsDf

,year,school,games,rushAtt,rushYds,rushAvg,rushTD,rec,recYds,recAvg,...,passAtt,compPct,passYds,ydsPerAtt,adjYdsPerAtt,TD,int,effRate,name,DraftYear
4,Career,Miami (FL),45.0,125,-320,-2.6,8,1.0,13.0,13.0,...,674.0,61.3,6058.0,9.0,8.7,48.0,25.0,152.9,Vinny Testaverde,1987.0
8,Career,Colorado State,32.0,141,-476,-3.4,2,NaN,NaN,NaN,...,1015.0,56.8,7142.0,7.0,6.1,36.0,38.0,120.2,Kelly Stouffer,1987.0
13,Career,Oregon,36.0,180,160,0.9,7,3.0,27.0,9.0,...,1015.0,55.2,6681.0,6.6,5.7,42.0,39.0,116.4,Chris Miller,1987.0
18,Career,Michigan,49.0,217,296,1.4,12,1.0,4.0,4.0,...,620.0,62.4,5449.0,8.8,8.2,31.0,22.0,145.6,Jim Harbaugh,1987.0
23,Career,Baylor,33.0,195,424,2.2,4,NaN,NaN,NaN,...,705.0,51.9,5411.0,7.7,6.4,33.0,34.0,122.2,Cody Carlson,1987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1767,Career,Overall,29.0,116,-131,-1.1,6,0.0,0.0,NaN,...,797.0,62.2,5630.0,7.1,7.3,43.0,15.0,135.6,Jake Luton,2020.0
1773,Career,Hawaii,33.0,251,880,3.5,12,0.0,0.0,NaN,...,1004.0,61.4,8032.0,8.0,8.3,70.0,24.0,146.8,Cole McDonald,2020.0
1777,Career,Pitt,11.0,57,144,2.5,1,0.0,0.0,NaN,...,167.0,54.5,1107.0,6.6,5.5,6.0,7.0,113.6,Ben DiNucci,2020.0
1783,Career,Overall,29.0,159,887,5.6,12,14.0,62.0,4.4,...,202.0,59.9,1459.0,7.2,7.4,15.0,6.0,139.1,Tommy Stevens,2020.0


In [6]:
# commit to sqlite database
cfbCareerStatsDf.to_sql('CFBStats', con=engine, if_exists='replace', index=False)